In [98]:
import csv

In [97]:
def get_domains(data):
    d=[set() for i in data[0]]
    for x in data:
        for i,value in enumerate(x):
            d[i].add(value)
    return list(sorted(x) for x in d)

In [99]:
def consistent(hypothesis, sample):
    more_general_parts = []
    for x, y in zip(hypothesis, sample):
        mg = x == "?" or (x != "0" and (x == y or y == "0"))
        more_general_parts.append(mg)
    return all(more_general_parts)

In [100]:
def min_generalization(hypothesis, sample):
    new_hypothesis = list(hypothesis)
    for i, key in enumerate(hypothesis):
        if not consistent(hypothesis[i], sample[i]):
            new_hypothesis[i] = '?' if hypothesis[i] != '0' else sample[i]
    return [tuple(new_hypothesis)]
    

In [101]:
def min_specialization(hypothesis, sample, domain):
    hypothesis = list(hypothesis)
    results = []
    for i, key in enumerate(hypothesis):
        if hypothesis[i] == '?':
            for val in domain[i]:
                if sample[i] != val:
                    new_hypothesis = hypothesis.copy()
                    new_hypothesis[i] = val
                    results.append(tuple(new_hypothesis))
        elif hypothesis[i] == '0':
            new_hypothesis = hypothesis.copy()
            new_hypothesis[i] = '0'
            results.append(tuple(new_hypothesis))
    return results
            

In [103]:
dataset=[]
with open('trainingdata.csv') as csvfile:
    lines=csv.reader(csvfile)
    for rows in list(lines)[1:]:
        dataset.append(rows)
print(dataset)

[['Sunny', 'Warm', 'Normal', 'Strong', 'Warm', 'Same', 'Yes'], ['Sunny', 'Warm', 'High', 'Strong', 'Warm', 'Same', 'Yes'], ['Rainy', 'Cold', 'High', 'Strong', 'Warm', 'Change', 'No'], ['Sunny', 'Warm', 'High', 'Strong', 'Cool', 'Change', 'Yes']]


In [104]:
domains=get_domains(dataset)
domains=domains[:-1]
print(domains)

[['Rainy', 'Sunny'], ['Cold', 'Warm'], ['High', 'Normal'], ['Strong'], ['Cool', 'Warm'], ['Change', 'Same']]


In [105]:
G={("?",)*len(domains)}
S={('0',)*len(domains)}
print(G,S)

{('?', '?', '?', '?', '?', '?')} {('0', '0', '0', '0', '0', '0')}


In [106]:
k=0
for i in dataset:
    k += 1
    attributes, output = i[:-1], i[-1]
    if output == 'Yes':
        G = {g for g in G if consistent(g, attributes)}
        for s in list(S):
            if not consistent(s, attributes):
                S.remove(s)
                s_plus = min_generalization(s, attributes)
                S.update([h for h in s_plus if any([consistent(g, h) for g in G])])
    else:
        S = {s for s in S if not consistent(s, attributes)}
        for g in list(G):
            if consistent(g, attributes):
                G.remove(g)
                g_minus = min_specialization(g, attributes, domains)
                G.update([h for h in g_minus if any([consistent(h, s) for s in S])])

    print("\n G[{0}]:".format(k), G)
    print("\n S[{0}]:".format(k), S)
            


 G[1]: {('?', '?', '?', '?', '?', '?')}

 S[1]: {('Sunny', 'Warm', 'Normal', 'Strong', 'Warm', 'Same')}

 G[2]: {('?', '?', '?', '?', '?', '?')}

 S[2]: {('Sunny', 'Warm', '?', 'Strong', 'Warm', 'Same')}

 G[3]: {('Sunny', '?', '?', '?', '?', '?'), ('?', 'Warm', '?', '?', '?', '?'), ('?', '?', '?', '?', '?', 'Same')}

 S[3]: {('Sunny', 'Warm', '?', 'Strong', 'Warm', 'Same')}

 G[4]: {('Sunny', '?', '?', '?', '?', '?'), ('?', 'Warm', '?', '?', '?', '?')}

 S[4]: {('Sunny', 'Warm', '?', 'Strong', '?', '?')}
